💡 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐨𝐭𝐚𝐥 𝐭𝐢𝐦𝐞 𝐬𝐩𝐞𝐧𝐭 𝐩𝐞𝐫 𝐬𝐞𝐬𝐬𝐢𝐨𝐧?
💡 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐨𝐭𝐚𝐥 𝐬𝐞𝐬𝐬𝐢𝐨𝐧 𝐭𝐢𝐦𝐞 𝐩𝐞𝐫 𝐮𝐬𝐞𝐫?

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,TimestampType
from datetime import datetime
from pyspark.sql.window import Window
from pyspark.sql.functions import col,unix_timestamp,lag,when,expr

In [0]:
data = [
    ("user1", "2023-08-26 10:10:00"),
    ("user1", "2023-08-26 10:10:25"),
    ("user2", "2023-08-26 12:00:00"),
    ("user2", "2023-08-26 12:10:00"),
    ("user1", "2023-08-26 14:30:00"),
    ("user1", "2023-08-26 16:00:00"),
    ("user2", "2023-08-26 16:30:00"),
    ("user1", "2023-08-26 18:00:00"),
]

In [0]:
schema = StructType([
    StructField("userid", StringType(), nullable=False),
    StructField("timestamp", TimestampType(), nullable=False),
])

strptime() is another method available in DateTime which is used to format the time stamp which is in string format to date-time object.

Syntax: datetime.strptime(time_data, format_data)

Parameter:

time_data is the time present in string format
format_data is the data present in datetime format which is converted from time_data using this function.

In [0]:
rows=[(row[0],datetime.strptime(row[1],"%Y-%m-%d %H:%M:%S")) for row in data]

In [0]:
df=spark.createDataFrame(rows,schema=schema)

In [0]:
df.show()

+------+-------------------+
|userid|          timestamp|
+------+-------------------+
| user1|2023-08-26 10:10:00|
| user1|2023-08-26 10:10:25|
| user2|2023-08-26 12:00:00|
| user2|2023-08-26 12:10:00|
| user1|2023-08-26 14:30:00|
| user1|2023-08-26 16:00:00|
| user2|2023-08-26 16:30:00|
| user1|2023-08-26 18:00:00|
+------+-------------------+



In [0]:
window_spec=Window().partitionBy("userid").orderBy("timestamp")

In [0]:
df=df.withColumn("session_gap",unix_timestamp("timestamp") - unix_timestamp(lag("timestamp").over(window_spec)))

In [0]:
df.show()

+------+-------------------+-----------+
|userid|          timestamp|session_gap|
+------+-------------------+-----------+
| user1|2023-08-26 10:10:00|       null|
| user1|2023-08-26 10:10:25|         25|
| user1|2023-08-26 14:30:00|      15575|
| user1|2023-08-26 16:00:00|       5400|
| user1|2023-08-26 18:00:00|       7200|
| user2|2023-08-26 12:00:00|       null|
| user2|2023-08-26 12:10:00|        600|
| user2|2023-08-26 16:30:00|      15600|
+------+-------------------+-----------+



In [0]:
df=df.withColumn("sessionid",when(col("session_gap") > 30, None).otherwise(col("userid") + expr("'_session'")+ expr("cast(row_number() over (partition by userid order by timestamp) as string)")))

In [0]:
df.show()

+------+-------------------+-----------+---------+
|userid|          timestamp|session_gap|sessionid|
+------+-------------------+-----------+---------+
| user1|2023-08-26 10:10:00|       null|     null|
| user1|2023-08-26 10:10:25|         25|     null|
| user1|2023-08-26 14:30:00|      15575|     null|
| user1|2023-08-26 16:00:00|       5400|     null|
| user1|2023-08-26 18:00:00|       7200|     null|
| user2|2023-08-26 12:00:00|       null|     null|
| user2|2023-08-26 12:10:00|        600|     null|
| user2|2023-08-26 16:30:00|      15600|     null|
+------+-------------------+-----------+---------+



In [0]:
#calculate totoal time spent per session
session_window=Window().partitionBy("userid","sessionid").orderBy("timestamp")
df=df.withColumn("session_start",when(col("session_gap") > 30,None).otherwise(lag("timestamp").over(session_window)))

In [0]:
df.show()

+------+-------------------+-----------+---------+-------------------+
|userid|          timestamp|session_gap|sessionid|      session_start|
+------+-------------------+-----------+---------+-------------------+
| user1|2023-08-26 10:10:00|       null|     null|               null|
| user1|2023-08-26 10:10:25|         25|     null|2023-08-26 10:10:00|
| user1|2023-08-26 14:30:00|      15575|     null|               null|
| user1|2023-08-26 16:00:00|       5400|     null|               null|
| user1|2023-08-26 18:00:00|       7200|     null|               null|
| user2|2023-08-26 12:00:00|       null|     null|               null|
| user2|2023-08-26 12:10:00|        600|     null|               null|
| user2|2023-08-26 16:30:00|      15600|     null|               null|
+------+-------------------+-----------+---------+-------------------+



In [0]:
df=df.withColumn("session_duration",when(col("session_start").isNotNull(),(col("timestamp").cast("long") - col("session_start").cast("long"))).otherwise(0))

In [0]:
df.show()

+------+-------------------+-----------+---------+-------------------+----------------+
|userid|          timestamp|session_gap|sessionid|      session_start|session_duration|
+------+-------------------+-----------+---------+-------------------+----------------+
| user1|2023-08-26 10:10:00|       null|     null|               null|               0|
| user1|2023-08-26 10:10:25|         25|     null|2023-08-26 10:10:00|              25|
| user1|2023-08-26 14:30:00|      15575|     null|               null|               0|
| user1|2023-08-26 16:00:00|       5400|     null|               null|               0|
| user1|2023-08-26 18:00:00|       7200|     null|               null|               0|
| user2|2023-08-26 12:00:00|       null|     null|               null|               0|
| user2|2023-08-26 12:10:00|        600|     null|               null|               0|
| user2|2023-08-26 16:30:00|      15600|     null|               null|               0|
+------+-------------------+----

In [0]:
#calculate total session time per user
user_window=Window().partitionBy("userid")
df=df.withColumn("total_user_session_time",expr("sum(session_duration) over (partition bu userid)"))
df.show()

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
<command-3181674450222561> in <cell line: 3>()
      1 #calculate total session time per user
      2 user_window=Window().partitionBy("userid")
----> 3 df=df.withColumn("total_user_session_time",expr("sum(session_duration) over (partition bu userid)"))
      4 df.show()

/databricks/spark/python/pyspark/sql/functions.py in expr(str)
   1705     [Row(length(name)=5), Row(length(name)=3)]
   1706     """
-> 1707     return _invoke_function("expr", str)
   1708 
   1709 

/databricks/spark/python/pyspark/sql/functions.py in _invoke_function(name, *args)
     85     assert SparkContext._active_spark_context is not None
     86     jf = _get_jvm_function(name, SparkContext._active_spark_context)
---> 87     return Column(jf(*args))
     88 
     89 

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *